In [1]:
import pandas as pd
import numpy as np
import copy
import time
import datetime
import holidays
from gekko import GEKKO

In [2]:
#calculamos el DeltaBOT y centramos en 0. Los objetos DF se llaman por referencia, por lo tanto se llama la direccion de mem del df original
#y por lo tanto todos los cambios serna reflejados en el df original
def ModifyDf(df): #recibe el dataframe
    #restamos el siguiente con el anterior de col BOT y dejamos en col DeltaBOT
    df['dBOT'] = df['BOT'] - df['BOT'].shift(1)
    #obtenemos el valor minimo de la columna
    dBOT_min = df['dBOT'].min()
    absmin = abs(dBOT_min)
    #llenamos los valores existentes con el minimo valor calculado anteriormente
    df['dBOT'] = df['dBOT'].fillna(dBOT_min)
    #sumamos el minimo a todas las filas para que valor minimo sea 0
    df['dBOT']+=absmin
    #pasamos la columna de fecha a datetime
    df['fecha_gestion'] =  pd.to_datetime(df['fecha_gestion'], format='%Y-%m-%d') 
     

#retornamos lista de dias no escogibles dentro de los proporcionados en el dataframe
def DefineBadDays(df, unique_dates_str):
    #obtenemos feriados de Chile
    cl_holidays = holidays.Chile()
    #creamos lista con dias que no se debe llamar
    bad_days=[]
    #agregamos las malas fechas a la lista bad_days
    for date_unique in unique_dates_str:
        date_time_obj = datetime.datetime.strptime(date_unique, '%Y-%m-%d')
        if date_unique in cl_holidays or date_time_obj.weekday() == 6:
            bad_days.append(pd.to_datetime(date_unique))
    return bad_days

#setemos recompensa muy negativa a los dias domingos o feriados
def SetDaysParams(df, bad_days): 
    #enmascaramos todos los valores donde la fecha sea conflictiva (con ~ se niega la condicion)
    filtered = df['fecha_gestion'].isin(bad_days)
    #usamos la mascara para modificar los valores
    df.loc[filtered,'dBOT']=-99
    
#funcion de optimizacion
def SetOptimizerOptions(): 
    m = GEKKO(remote=False) # Initialize gekko
    m.options.SOLVER=1  # APOPT is an MINLP solver
    m.solver_options = ['minlp_maximum_iterations 1000', \
                        # minlp iterations with integer solution
                        'minlp_max_iter_with_int_sol 100', \
                        # treat minlp as nlp
                        'minlp_as_nlp 1', \
                        # nlp sub-problem max iterations
                        'nlp_maximum_iterations 250', \
                        # 1 = depth first, 2 = breadth first
                        'minlp_branch_method 1', \
                        # maximum deviation from whole number
                        'minlp_integer_tol 0.001', \
                        # covergence tolerance
                        'minlp_gap_tol 0.001']
    m.options.REDUCE=3
    return m

#checkea si fecha actual es domingo o festivo
def check_dates(bad_days, actual_day):
    flag = False
    for bad_date in bad_days:
        if actual_day == bad_date:
            flag = True
    return flag
    
#recibe ela recomenza (px_params) de usuario por rut unico y as fechas unicas y bad days
def Optimizer(px_params, unique_dates, bad_days, m, n_vars):
    cnt = 0
    user_dates={}
    start_all = time.time()
    for ix_dates, item_dates in enumerate(unique_dates, start=0):
        if cnt >= 100: #caso extremo que no debebiese suceder
            break
        #cheackeamos si es un festivo o domingo y lo saltamos de ser el caso
        isbadDate = check_dates(bad_days, unique_dates[ix_dates])
        if(isbadDate):
            print('dia: ',unique_dates[ix_dates], 'es feriado o domingo')
            cnt+=1
            continue
        else:
            #variable para ir reduciendo el numero de variables (optimizacion de codido) y asi resolver problema mas pequeño
            n_vars2 = n_vars  - cnt
            start_loop = time.time()
            print('calculando dia: ',np.datetime_as_string(unique_dates[ix_dates],'D'))
            #arreglo de variables (dias)
            x_vars=m.Array(m.Var, (n_vars2), integer=True,lb=0,ub=1)
            #restriccion de a lo mas 2 llamadas por mes
            #m.Equation(sum(x_vars)<=2)
            m.Equation(m.sum(x_vars)<=2)
            #restriccion que indica que puede llamar cada 7 dias, para todos los dias
            for ix_x, itemx in enumerate(x_vars):
                m.Equation(m.sum(x_vars[ix_x:ix_x+7])<=1)
                #m.Equation(sum(x_vars[ix_x:ix_x+7])<=1)
            #funcion objetivo (notar cnt: significa toma la el arreglo, desde la posicion actual 'cnt' en adelante)
            #m.Maximize(sum(x_vars[idx]*px_params_copy[idx+cnt] for idx in range(n_vars2)))
            m.Maximize(m.sum(x_vars*px_params[cnt:] ))
            #resolucion
            m.solve(disp=False,debug=False)
            #lista de dias posibles de llamado por dia
            idx_choosen = []
            #escribimos los dias posibles (notar ese + cnt ya que optimizamos con - cnt variables)
            #por lo que hay que mover la solucion teporalmente hacia el futuro
            for ix_xx, itemx in enumerate(x_vars):
                #print(itemx.value)
                if itemx.value[0] == 1:
                    idx_choosen.append(np.datetime_as_string(unique_dates[ix_xx + cnt],'D'))
            user_dates[np.datetime_as_string(unique_dates[ix_dates],unit='D')]=idx_choosen
            #px_params_copy[ix_dates] = -50
            end_loop = time.time()
            print('looped time: ',end_loop - start_loop,' [s]',' total: ', end_loop - start_all,' [s]', 
                  (end_loop - start_all)/60,' [m] cantidad de variables: ',len(x_vars))
            cnt+=1
    return user_dates

In [3]:
def TestFunc():
    rec_dev = pd.read_csv("recomendacion_dev.csv")
    #se calculan date unique con str y datetime64 por separado (antes y despues)
    unique_dates_str = rec_dev['fecha_gestion'].unique()
    ModifyDf(rec_dev)
    unique_dates = rec_dev['fecha_gestion'].unique()
    unique_ruts = rec_dev['rut_num'].unique()
    n_vars=len(unique_dates)
    bad_days = DefineBadDays(rec_dev, unique_dates_str)
    SetDaysParams(rec_dev, bad_days)
    m = SetOptimizerOptions()
    
    #aca se podria loopear por unique ruts o bien pasar el el rut a TestFunc y loopear por afuera y pasale rut
    #para calcular los paramtros del usuario
    px_params = rec_dev[rec_dev["rut_num"] == unique_ruts[0]]['dBOT'].to_numpy()
    user_dates = Optimizer(px_params, unique_dates, bad_days, m, n_vars)
    #########################################
    #limpiar GEKKO APmonitor
    m.cleanup()
    
    #salida, acumular en una lista de user_dates, escribir luego en csv, esbirir en una base de datos... etc.
    print(' ')
    print(user_dates)
    

In [4]:
TestFunc()

calculando dia:  2021-06-01
looped time:  0.30216050148010254  [s]  total:  0.30216050148010254  [s] 0.005036008358001709  [m] cantidad de variables:  30
calculando dia:  2021-06-02
looped time:  0.6300725936889648  [s]  total:  0.9329330921173096  [s] 0.015548884868621826  [m] cantidad de variables:  29
calculando dia:  2021-06-03
looped time:  0.928903341293335  [s]  total:  1.8628349304199219  [s] 0.031047248840332033  [m] cantidad de variables:  28
calculando dia:  2021-06-04
looped time:  1.0091323852539062  [s]  total:  2.8729636669158936  [s] 0.04788272778193156  [m] cantidad de variables:  27
calculando dia:  2021-06-05
looped time:  1.7380881309509277  [s]  total:  4.611051797866821  [s] 0.07685086329778036  [m] cantidad de variables:  26
dia:  2021-06-06T00:00:00.000000000 es feriado o domingo
calculando dia:  2021-06-07
looped time:  1.919295072555542  [s]  total:  6.532420635223389  [s] 0.10887367725372314  [m] cantidad de variables:  24
calculando dia:  2021-06-08
looped t